<b>Memoization позволяет сделать кэш функции. Так функция работает намного быстрее</b>

In [2]:
def fibonacci(n):
    print("Calculation fibonacci({0})".format(n))
    return 1 if n < 3 else fibonacci(n - 1) + fibonacci(n - 2)


print(fibonacci(10))
#так неэффективно, слишком много возвратов функции, это замедляет её работу

Calculation fibonacci(10)
Calculation fibonacci(9)
Calculation fibonacci(8)
Calculation fibonacci(7)
Calculation fibonacci(6)
Calculation fibonacci(5)
Calculation fibonacci(4)
Calculation fibonacci(3)
Calculation fibonacci(2)
Calculation fibonacci(1)
Calculation fibonacci(2)
Calculation fibonacci(3)
Calculation fibonacci(2)
Calculation fibonacci(1)
Calculation fibonacci(4)
Calculation fibonacci(3)
Calculation fibonacci(2)
Calculation fibonacci(1)
Calculation fibonacci(2)
Calculation fibonacci(5)
Calculation fibonacci(4)
Calculation fibonacci(3)
Calculation fibonacci(2)
Calculation fibonacci(1)
Calculation fibonacci(2)
Calculation fibonacci(3)
Calculation fibonacci(2)
Calculation fibonacci(1)
Calculation fibonacci(6)
Calculation fibonacci(5)
Calculation fibonacci(4)
Calculation fibonacci(3)
Calculation fibonacci(2)
Calculation fibonacci(1)
Calculation fibonacci(2)
Calculation fibonacci(3)
Calculation fibonacci(2)
Calculation fibonacci(1)
Calculation fibonacci(4)
Calculation fibonacci(3)

#### Создадим класс с кешем

In [3]:
class MyFib(object):
    def __init__(self):
        self.cache = {1: 1, 2: 1}

    def fibo(self, n):
        if n not in self.cache:
            print("Calculation ({})".format(n))
            self.cache[n] = self.fibo(n - 1) + self.fibo(n - 2)
        return self.cache[n]


a = MyFib()
print(a.fibo(10))

Calculation (10)
Calculation (9)
Calculation (8)
Calculation (7)
Calculation (6)
Calculation (5)
Calculation (4)
Calculation (3)
55


### Сделаем тоже самое через замыкание

In [4]:
def clousure_cache():
    cache = {1: 1, 2: 1}

    def inner(n):
        if n not in cache:
            print("Calculation ({0})".format(n))
            cache[n] = inner(n - 1) + inner(n - 2)
        return cache[n]
    return inner


print(clousure_cache()(10))

Calculation (10)
Calculation (9)
Calculation (8)
Calculation (7)
Calculation (6)
Calculation (5)
Calculation (4)
Calculation (3)
55


### Сделаем такое же кеширование с помощью декоратора

In [18]:
#работает для любой функции, которая принимает единичный параметр

def memoize(fn):
    cache = dict() #нам не нужно задавать начальные условия, если n нет в cache, то n туда добавиться

    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    return inner


@memoize
def fibo_n(n):
    print("Calculating ({})".format(n))
    return 1 if n < 3 else fibo_n(n - 1) + fibo_n(n - 2)


print(fibo_n(10))

Calculating (10)
Calculating (9)
Calculating (8)
Calculating (7)
Calculating (6)
Calculating (5)
Calculating (4)
Calculating (3)
Calculating (2)
Calculating (1)
55


In [11]:
@memoize
def fact(n):
    print("Calculating ({})!".format(n))
    return 1 if n < 2 else n * fact(n-1)

print(fact(6))

Calculating (6)!
Calculating (5)!
Calculating (4)!
Calculating (3)!
Calculating (2)!
Calculating (1)!
720


In [12]:
#если вызовем fact(6), функция не будет считать снова сначала, а возмём значение из кеша
print(fact(6))
print(fact(7))

720
Calculating (7)!
5040


### Посмотрим скорость работы

In [17]:
from time import perf_counter

start = perf_counter()

print(fibo_n(36))
print("{:.6f}".format(perf_counter() - start))

14930352
0.000981


### Для кеширования можем использовать встроенный модуль lru_cache пакета functools

In [2]:
from functools import lru_cache


@lru_cache(maxsize = 8)
def fibo(n):
    print("Calculation ({})".format(n))
    return 1 if n < 3 else fibo(n - 1) + fibo(n - 2)


print(fibo(8))

Calculation (8)
Calculation (7)
Calculation (6)
Calculation (5)
Calculation (4)
Calculation (3)
Calculation (2)
Calculation (1)
21


У lru_cache есть аргумент maxsize (максимальный размер кеша). По умолчанию maxsize = 128,
но можем поставить maxsize = None для теоретически безразмерного кеша. Хорошая практика
задавать maxsize кратный 2. Если задан размер кеша (maxsize), то кеш всегда равен этому размеру,
т.е. высчитывается функция, но в кеше остаётся только то кол-во результатов, прописанных в maxsize,
остальные вытесняются.